# Preprocessing

In [2]:
import pandas as pd
df = pd.read_csv('../data/stroke.csv')

In [5]:
df.shape

(43400, 12)

In [6]:
df.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [15]:
df = df.dropna()
df.shape

(29072, 12)

In [25]:
from sklearn.preprocessing import OneHotEncoder
onehot_enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
df_onehot = df[['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']]
onehot_encoded = onehot_enc.fit_transform(df_onehot)
df_onehot = pd.DataFrame(onehot_encoded, columns = onehot_enc.get_feature_names())

In [24]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
df_minmax = df[['age', 'avg_glucose_level', 'bmi']]
minmax_encoded = minmax_scaler.fit_transform(df_minmax)
df_minmax = pd.DataFrame(minmax_encoded, columns = ['age', 'avg_glucose_level', 'bmi'])

In [26]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
label_encoded = label_enc.fit_transform(df['stroke'])
df_label = pd.DataFrame(label_encoded, columns = ['stroke'])

In [36]:
df_preprocessed = pd.concat([df_onehot, df_minmax, df_label], axis = 1)

(29072, 23)

In [37]:
df_preprocessed.shape

(29072, 23)

In [38]:
df_preprocessed.columns

Index(['x0_Female', 'x0_Male', 'x0_Other', 'x1_0', 'x1_1', 'x2_0', 'x2_1',
       'x3_No', 'x3_Yes', 'x4_Govt_job', 'x4_Never_worked', 'x4_Private',
       'x4_Self-employed', 'x4_children', 'x5_Rural', 'x5_Urban',
       'x6_formerly smoked', 'x6_never smoked', 'x6_smokes', 'age',
       'avg_glucose_level', 'bmi', 'stroke'],
      dtype='object')

In [39]:
df_preprocessed.to_csv('../data/stroke_preprocessed.csv')